In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25058
2,Huelva,Confirmados PDIA 14 días,382
3,Huelva,Tasa PDIA 14 días,"74,43926963774187"
4,Huelva,Confirmados PDIA 7 días,232
5,Huelva,Total Confirmados,25257
6,Huelva,Curados,19253
7,Huelva,Fallecidos,346


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25058.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6633.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3688 personas en los últimos 7 días 

Un positivo PCR cada 2247 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25058.0,232.0,382.0,513170.0,45.209190,74.439270,59.0
Condado-Campiña,7110.0,134.0,198.0,156231.0,85.770430,126.735411,47.0
Huelva-Costa,15226.0,89.0,172.0,289548.0,30.737563,59.402931,12.0
Almonte,916.0,37.0,48.0,24507.0,150.977272,195.862407,11.0
Moguer,860.0,24.0,31.0,21867.0,109.754424,141.766132,11.0
Huelva (capital),6633.0,39.0,64.0,143837.0,27.114025,44.494810,10.0
Gibraleón,530.0,15.0,19.0,12737.0,117.767135,149.171704,9.0
Rociana del Condado,323.0,10.0,12.0,7939.0,125.960448,151.152538,3.0
Bollullos Par del Condado,634.0,13.0,18.0,14387.0,90.359352,125.112949,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,108.0,6.0,13.0,3092.0,194.049159,420.439845,0.0
Villalba del Alcor,458.0,7.0,13.0,3366.0,207.961973,386.215092,2.0
San Bartolomé de la Torre,212.0,8.0,13.0,3761.0,212.709386,345.652752,2.0
Almonte,916.0,37.0,48.0,24507.0,150.977272,195.862407,11.0
Alosno,128.0,5.0,6.0,3933.0,127.129418,152.555301,0.0
Rociana del Condado,323.0,10.0,12.0,7939.0,125.960448,151.152538,3.0
Gibraleón,530.0,15.0,19.0,12737.0,117.767135,149.171704,9.0
Moguer,860.0,24.0,31.0,21867.0,109.754424,141.766132,11.0
Villarrasa,132.0,2.0,3.0,2211.0,90.456807,135.685210,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,1233.0,4.0,27.0,21104.0,18.953753,127.937832,1.0,0.148148
San Juan del Puerto,368.0,3.0,9.0,9411.0,31.877590,95.632770,1.0,0.333333
Palma del Condado (La),988.0,4.0,10.0,10801.0,37.033608,92.584020,0.0,0.400000
Puebla de Guzmán,108.0,6.0,13.0,3092.0,194.049159,420.439845,0.0,0.461538
Escacena del Campo,141.0,1.0,2.0,2287.0,43.725404,87.450809,0.0,0.500000
Huelva-Costa,15226.0,89.0,172.0,289548.0,30.737563,59.402931,12.0,0.517442
Isla Cristina,2004.0,10.0,19.0,21393.0,46.744262,88.814098,0.0,0.526316
Villalba del Alcor,458.0,7.0,13.0,3366.0,207.961973,386.215092,2.0,0.538462
Palos de la Frontera,461.0,5.0,9.0,11742.0,42.582184,76.647931,1.0,0.555556
